ref: https://d3rlpy.readthedocs.io/en/stable/tutorials/offline_policy_selection.html

# Offline Policy Selection

d3rlpy supports offline policy selection by training Fitted Q Evaluation (FQE), which is an offline on-policy RL algorithm. The use of FQE for offline policy selection is proposed by Paine et al.. The concept is that FQE trains Q-function with the trained policy in on-policy manner so that the learned Q-function reflects the expected return of the trained policy. By using the Q-value estimation of FQE, the candidate trained policies can be ranked only with offline dataset. Check Off-Policy Evaluation for more information.

>Offline policy selection with FQE is confirmed that it usually works out with discrete action-space policies. However, it seems require some hyperparameter tuning for ranking continuous action-space policies. The more techniques will be supported along with the advancement of this research domain.

## Prepare trained policies

In this tutorial, let’s train DQN with the built-in CartPole-v0 dataset.

In [8]:
import d3rlpy

# setup replay CartPole-v0 dataset and environment
dataset, env = d3rlpy.datasets.get_dataset("cartpole-replay")

# setup algorithm

dqn = d3rlpy.algos.DQNConfig().create()

import gymnasium as gym
# start offline training
dqn.fit(
   dataset,
   n_steps=100000,
   n_steps_per_epoch=10000,
   evaluators={
       "environment": d3rlpy.metrics.EnvironmentEvaluator(env),
   },
)

2025-09-15 23:21.38 [info     ] Signatures have been automatically determined. action_signature=Signature(dtype=[dtype('int32')], shape=[(1,)]) observation_signature=Signature(dtype=[dtype('float32')], shape=[(4,)]) reward_signature=Signature(dtype=[dtype('float32')], shape=[(1,)])
2025-09-15 23:21.38 [info     ] Action-space has been automatically determined. action_space=<ActionSpace.DISCRETE: 2>
2025-09-15 23:21.38 [info     ] Action size has been automatically determined. action_size=2
2025-09-15 23:21.39 [info     ] dataset info                   dataset_info=DatasetInfo(observation_signature=Signature(dtype=[dtype('float32')], shape=[(4,)]), action_signature=Signature(dtype=[dtype('int32')], shape=[(1,)]), reward_signature=Signature(dtype=[dtype('float32')], shape=[(1,)]), action_space=<ActionSpace.DISCRETE: 2>, action_size=2)
2025-09-15 23:21.39 [debug    ] Building models...            
2025-09-15 23:21.39 [debug    ] Models have been built.       
2025-09-15 23:21.39 [info    

Epoch 1/10: 100%|██████████| 10000/10000 [00:09<00:00, 1089.14it/s, loss=0.00657]


AttributeError: module 'numpy' has no attribute 'bool8'

In [13]:
import d3rlpy

# setup the same dataset used in policy training
dataset, _ = d3rlpy.datasets.get_dataset("cartpole-replay")

# load pretrained policy
dqn = d3rlpy.load_learnable("d3rlpy_logs/DQN_20250915221802/model_10000.d3")

# setup FQE algorithm
fqe = d3rlpy.ope.DiscreteFQE(algo=dqn, config=d3rlpy.ope.DiscreteFQEConfig())

# start FQE training
fqe.fit(
   dataset,
   n_steps=10000,
   n_steps_per_epoch=1000,
   scorers={
       "init_value": d3rlpy.metrics.InitialStateValueEstimationEvaluator(),
       "soft_opc": d3rlpy.metrics.SoftOPCEvaluator(180),  # set 180 for success return threshold here
   },
)

2025-09-15 23:23.28 [info     ] Signatures have been automatically determined. action_signature=Signature(dtype=[dtype('int32')], shape=[(1,)]) observation_signature=Signature(dtype=[dtype('float32')], shape=[(4,)]) reward_signature=Signature(dtype=[dtype('float32')], shape=[(1,)])
2025-09-15 23:23.28 [info     ] Action-space has been automatically determined. action_space=<ActionSpace.DISCRETE: 2>
2025-09-15 23:23.28 [info     ] Action size has been automatically determined. action_size=2


AttributeError: module 'd3rlpy.ope' has no attribute 'DiscreteFQEConfig'